# SAS 프로그램 결과

In [25]:
import saspy #SAS출력 코드

winlocal 입력

## 9장

In [26]:
%%SAS

libname ex 'C:\Biostat';
run;

/*9장 연습문제 불러오기*/
%macro chap09(name=,no=);
%do i=1 %to &no.;
	PROC IMPORT DBMS=excel
		DATAFILE="C:\Biostat\data_chap09"
		OUT=ex.&name.&i. REPLACE;
		RANGE="exam9_&i.$";
	RUN;
%end;
%mend;

%chap09(name=ex9_,no=2);

### EXAMPLE 9.1

In [27]:
%%SAS

DATA ex9_1_diff;
    SET ex.ex9_1;
    diff=hindleg-foreleg;
    abs_diff = abs(diff);
RUN;

ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex9_1_diff normal;
    VAR diff;
ods output TestsForNormality = TestsForNormality;
RUN;

PROC TTEST DATA=ex.ex9_1;
	PAIRED hindleg * foreleg ;
    ods output TTests =TTests;
RUN;
ods graphics on;ods exclude none;ods results;

PROC SORT DATA=TestsForNormality;
    BY descending Test;
RUN;
title 'Example 9.1: 정규성검정';
PROC PRINT DATA=TestsForNormality label;
RUN;
title 'Example 9.1: Two-Tailed Paired-Sample t Test';
PROC PRINT DATA=TTests label;
RUN;

### EXAMPLE 9.2

In [28]:
%%SAS

ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex.ex9_2 normal;
    VAR d;
ods output TestsForNormality = TestsForNormality;
RU;
PROC TTEST order=data h0=250 SIDES=u
	DATA=ex.ex9_2;
	PAIRED new * old ;
    ods output TTests =TTests;
RUN;
ods graphics on;ods exclude none;ods results;

PROC SORT DATA=TestsForNormality;
 BY descending Test;
RUN;
title 'Example 9.2: 정규성검정';
PROC PRINT DATA=TestsForNormality label;
run;
title 'Example 9.2: One-Tailed Paired-Sample t Test';
PROC PRINT DATA=TTests label;
RUN;

### EXAMPLE 9.3

In [29]:
%%SAS

PROC IML;
    use ex.ex9_1 ; read all;
    n=10;
    s1=var(hindleg); 
	s2=var(foreleg);
	F=s1/s2;
	r=0.6739;
	t=((F-1)*sqrt(n-2))/(2*sqrt(F*(1-(r*r))));
	p=probt(-0.655, 8)*2;
PRINT F t p;
RUN;
QUIT;

F,t,p
0.7110656,-0.655872,0.530846


### EXAMPLE 9.4

In [30]:
%%SAS

ods graphics off;ods exclude all;ods noresults;
PROC UNIVARIATE DATA=ex9_1_diff;
     VAR diff;
ods output TestsForLocation = TestsForLocation;
RUN;
ods graphics on;ods exclude none;ods results;
PROC PRINT DATA=TestsForLocation label;
RUN;

OBS,VarName,위치모수 검정,검정 통계량 레이블,검정 통계량,P-값 설명,P-값,Mu0 값
1,diff,스튜던트의 t,t,3.413793,Pr > |t|,0.0077,0
2,diff,부호,M,3,Pr >= |M|,0.1094,0
3,diff,부호 순위,S,23.5,Pr >= |S|,0.0117,0


In [31]:
%%SAS

PROC RANK DATA=ex9_1_diff out=ex9_1_rank;
    VAR abs_diff;
	RANKS rank;
RUN;
DATA ex9_1_signed_rank;
	SET ex9_1_rank;
	if diff < 0 then signed_rank = rank*(-1);
	else if diff > 0 then  signed_rank = rank;
RUN;
PROC PRINT DATA=ex9_1_signed_rank;
RUN;

PROC IML;
	use ex9_1_signed_rank;
	read all;
	close ex9_1_signed_rank;
		n = nrow(deer);
		plus = loc(diff>0);
		minus = loc(diff<0);
		rank_plus = rank[plus];
		rank_minus = rank[minus];
		T_plus = sum(rank_plus);
		T_minus = sum(rank_minus);
	PRINT T_plus T_minus ;
RUN;
QUIT;

OBS,deer,hindleg,foreleg,diff,abs_diff,rank,signed_rank
1,1,142,138,4,4,4.5,4.5
2,2,140,136,4,4,4.5,4.5
3,3,144,147,-3,3,3.0,-3.0
4,4,144,139,5,5,7.0,7.0
5,5,142,143,-1,1,1.0,-1.0
6,6,146,141,5,5,7.0,7.0
7,7,149,143,6,6,9.5,9.5
8,8,150,145,5,5,7.0,7.0
9,9,142,136,6,6,9.5,9.5
10,10,148,146,2,2,2.0,2.0
